# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [111]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score   # useful for calculating percentile scores
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [113]:
stocks = pd.read_csv("sp_500_stocks_2.csv")  # csv.read to read csv files
from secrets import IEX_CLOUD_API_TOKEN    # get api token from secrets file

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [114]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [115]:
data['year1ChangePercent']

0.8446742115922801

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [116]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
# print(symbol_strings)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [117]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb,tsla&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'           
        ],
        index = my_columns),
        ignore_index = True
        )


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [118]:
# for this we will first sort the stock's obe year price return and then drop entries outside top 50

# to sort we can use pandas 'sort_values' function which takes 1.col name
# 2. Sorting order
# 3. inplace --- if inplace = True it will sort the same dataframe and return but if it is false it will return a copy of sorted dataframe which you can use
# inplace = True
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
# inplace = False
# final_dataframe = final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = False)

final_dataframe = final_dataframe[:50]  # only take 1st 50 rows
# final_dataframe.reset_index(inplace = True)  # to reset the index value that was unsorted
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.00,2.267452,N/A
1,341,NVDA,534.90,1.356925,N/A
2,31,AMD,101.27,1.350567,N/A
3,273,LB,40.30,1.248779,N/A
4,383,PYPL,241.30,1.117830,N/A
5,23,ALB,136.82,1.097816,N/A
6,24,ALGN,522.86,0.940665,N/A
7,335,NOW,549.00,0.939983,N/A
8,177,FCX,25.01,0.918464,N/A
9,3,AAPL,131.76,0.883043,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [125]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter your portfolio size: ")
    try:
        float(portfolio_size)
    except ValueError:
        print("Incorrect! Please try again with a valid integer")
        portfolio_size = input("Enter your portfolio size: ")

portfolio_input()
print(portfolio_size)

Enter your portfolio size: 
Incorrect! Please try again with a valid integer
Enter your portfolio size: 10000000
10000000


In [126]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
     
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.00,2.267452,5263
1,341,NVDA,534.90,1.356925,373
2,31,AMD,101.27,1.350567,1974
3,273,LB,40.30,1.248779,4962
4,383,PYPL,241.30,1.117830,828
5,23,ALB,136.82,1.097816,1461
6,24,ALGN,522.86,0.940665,382
7,335,NOW,549.00,0.939983,364
8,177,FCX,25.01,0.918464,7996
9,3,AAPL,131.76,0.883043,1517


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [182]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',    
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb,tsla&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data)
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
    
hqm_dataframe
    
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.05,N/A,0.410902,N/A,0.361768,N/A,0.182626,N/A,0.065787,N/A,N/A
1,AAL,17.30,N/A,-0.394808,N/A,-0.010011,N/A,0.232387,N/A,0.332391,N/A,N/A
2,AAP,166.10,N/A,0.059520,N/A,0.135396,N/A,0.046061,N/A,0.069478,N/A,N/A
3,AAPL,130.59,N/A,0.884269,N/A,0.475455,N/A,0.142658,N/A,0.062529,N/A,N/A
4,ABBV,105.80,N/A,0.245512,N/A,0.122777,N/A,0.187123,N/A,0.064583,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,107.47,N/A,0.085437,N/A,0.181462,N/A,0.141805,N/A,0.031115,N/A,N/A
497,ZBH,150.85,N/A,0.000224,N/A,0.141146,N/A,0.022683,N/A,-0.009171,N/A,N/A
498,ZBRA,392.93,N/A,0.478894,N/A,0.418437,N/A,0.497361,N/A,0.093494,N/A,N/A
499,ZION,43.55,N/A,-0.156360,N/A,0.153646,N/A,0.338034,N/A,0.055751,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [183]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

#now to change the values of percetile columns:
# print(hqm_dataframe)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col =  f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row,change_col])/100
                
hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.05,N/A,0.410902,0.868263,0.361768,0.814371,0.182626,0.656687,0.065787,0.736527,N/A
1,AAL,17.30,N/A,-0.394808,0.0279441,-0.010011,0.121756,0.232387,0.786427,0.332391,0.994012,N/A
2,AAP,166.10,N/A,0.059520,0.469062,0.135396,0.371257,0.046061,0.279441,0.069478,0.746507,N/A
3,AAPL,130.59,N/A,0.884269,0.982036,0.475455,0.902196,0.142658,0.54491,0.062529,0.722555,N/A
4,ABBV,105.80,N/A,0.245512,0.736527,0.122777,0.337325,0.187123,0.668663,0.064583,0.728543,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,107.47,N/A,0.085437,0.510978,0.181462,0.481038,0.141805,0.540918,0.031115,0.576846,N/A
497,ZBH,150.85,N/A,0.000224,0.377246,0.141146,0.381238,0.022683,0.219561,-0.009171,0.347305,N/A
498,ZBRA,392.93,N/A,0.478894,0.906188,0.418437,0.856287,0.497361,0.974052,0.093494,0.804391,N/A
499,ZION,43.55,N/A,-0.156360,0.165669,0.153646,0.403194,0.338034,0.908184,0.055751,0.696607,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [184]:
from statistics import mean

# now we iterate through each row and get return percentile value for all periods and get the mean of it which is our HQM score
for row in hqm_dataframe.index: 
    momentum_percentiles = []  
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
# hqm_dataframe

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [185]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.27,N/A,0.919330,0.986028,1.300183,0.998004,0.473168,0.962076,0.161902,0.906188,0.963074
1,ALGN,539.50,N/A,0.912822,0.984032,0.983941,0.992016,0.569815,0.982036,0.133312,0.876248,0.958583
2,TWTR,54.79,N/A,0.746569,0.958084,0.570339,0.946108,0.379067,0.928144,0.276113,0.982036,0.953593
3,LRCX,495.87,N/A,0.770848,0.964072,0.650203,0.964072,0.648326,0.992016,0.128762,0.870259,0.947605
4,XLNX,158.16,N/A,0.602095,0.944112,0.656021,0.966068,0.496977,0.972056,0.156476,0.9002,0.945609
5,AMAT,90.19,N/A,0.505671,0.914172,0.526285,0.936128,0.598364,0.984032,0.192877,0.936128,0.942615
6,AMD,98.77,N/A,1.340278,0.996008,0.798003,0.986028,0.273644,0.844311,0.160700,0.902196,0.932136
7,LB,39.70,N/A,1.210751,0.994012,1.417763,1,0.308152,0.874251,0.114628,0.848303,0.929142
8,PYPL,232.90,N/A,1.119911,0.992016,0.443295,0.886228,0.276894,0.850299,0.202790,0.952096,0.92016
9,MU,73.76,N/A,0.391832,0.858283,0.440454,0.884232,0.473485,0.964072,0.182737,0.928144,0.908683


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [192]:
portfolio_input()

Enter your portfolio size: 10000000


In [193]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])

hqm_dataframe

C:\Users\arvam\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.27,7914,0.919330,0.986028,1.300183,0.998004,0.473168,0.962076,0.161902,0.906188,0.963074
1,ALGN,539.50,370,0.912822,0.984032,0.983941,0.992016,0.569815,0.982036,0.133312,0.876248,0.958583
2,TWTR,54.79,3650,0.746569,0.958084,0.570339,0.946108,0.379067,0.928144,0.276113,0.982036,0.953593
3,LRCX,495.87,403,0.770848,0.964072,0.650203,0.964072,0.648326,0.992016,0.128762,0.870259,0.947605
4,XLNX,158.16,1264,0.602095,0.944112,0.656021,0.966068,0.496977,0.972056,0.156476,0.9002,0.945609
5,AMAT,90.19,2217,0.505671,0.914172,0.526285,0.936128,0.598364,0.984032,0.192877,0.936128,0.942615
6,AMD,98.77,2024,1.340278,0.996008,0.798003,0.986028,0.273644,0.844311,0.160700,0.902196,0.932136
7,LB,39.70,5037,1.210751,0.994012,1.417763,1,0.308152,0.874251,0.114628,0.848303,0.929142
8,PYPL,232.90,858,1.119911,0.992016,0.443295,0.886228,0.276894,0.850299,0.202790,0.952096,0.92016
9,MU,73.76,2711,0.391832,0.858283,0.440454,0.884232,0.473485,0.964072,0.182737,0.928144,0.908683


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [194]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [195]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [196]:
column_formats = {
        'A' : ['Ticker', string_template],
        'B' : ['Price',dollar_template],
        'C' : ['Number of Shares to Buy',integer_template],
        'D' : ['One-Year Price Return',percent_template],
        'E' : ['One-Year Return Percentile',percent_template],
        'F' : ['Six-Month Price Return',percent_template],
        'G' : ['Six-Month Return Percentile',percent_template],    
        'H' : ['Three-Month Price Return',percent_template],
        'I' : ['Three-Month Return Percentile',percent_template],
        'J' : ['One-Month Price Return',percent_template],
        'K' : ['One-Month Return Percentile',percent_template],
        'L' : ['HQM Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [197]:
writer.save()